In [ ]:
from fastapi import FastAPI, HTTPException
from py2neo import Graph
import pyodbc

app = FastAPI()

# SQL Server Connection
sql_server = 'mcruebs04.isad.isadroot.ex.ac.uk'
sql_database = 'BEMM459_GroupO'
sql_username = 'GroupO'
sql_password = 'MrjV827*Wr'

sql_conn = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={sql_server};DATABASE={sql_database};UID={sql_username};PWD={sql_password};',
    autocommit=True
)
sql_cursor = sql_conn.cursor()

# Neo4j Connection
neo4j_uri = "neo4j+s://dd0c088b.databases.neo4j.io"
neo4j_username = "neo4j"
neo4j_password = "lapDEOAyHjUWt31Bg123clw9s6949mg56APvryc4-tY"

neo4j_graph = Graph(neo4j_uri, auth=(neo4j_username, neo4j_password))

# API to fetch customer details from SQL and related parcels from Neo4j
@app.get("/customers/{customer_id}")
async def get_customer(customer_id: int):
    try:
        sql_cursor.execute("SELECT * FROM Customers WHERE CustomerID = ?", (customer_id,))
        customer = sql_cursor.fetchone()
        if not customer:
            raise HTTPException(status_code=404, detail="Customer not found")

        query = """
        MATCH (c:Customer {ID: $customer_id})-[:OWNS]->(p:Parcel)
        RETURN p.ID as ParcelID, p.Name as ParcelName, p.Weight as Weight, p.Type as Type
        """
        parcels = neo4j_graph.run(query, customer_id=str(customer_id)).data()

        return {
            "CustomerID": customer[0],
            "Name": customer[1],
            "Email": customer[2],
            "Phone": customer[3],
            "Address": customer[4],
            "Type": customer[5],
            "Parcels": parcels
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# API to fetch parcel details from SQL and related shipments from SQL
@app.get("/parcels/{parcel_id}")
async def get_parcel(parcel_id: int):
    try:
        sql_cursor.execute("SELECT * FROM Parcels WHERE ParcelID = ?", (parcel_id,))
        parcel = sql_cursor.fetchone()
        if not parcel:
            raise HTTPException(status_code=404, detail="Parcel not found")

        sql_cursor.execute("SELECT * FROM Shipments WHERE ParcelID = ?", (parcel_id,))
        shipments = sql_cursor.fetchall()

        shipment_list = [{
            "ShipmentID": shipment[0],
            "ShipmentName": shipment[1],
            "ShipmentDate": shipment[3],
            "DeliveryDate": shipment[4],
            "Status": shipment[5],
            "CurrentLocation": shipment[6]
        } for shipment in shipments]

        return {
            "ParcelID": parcel[0],
            "ParcelName": parcel[1],
            "CustomerID": parcel[2],
            "Weight": parcel[3],
            "Type": parcel[4],
            "ShippingMethod": parcel[5],
            "Shipments": shipment_list
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# API to fetch shipment details from SQL and related analytics from Neo4j
@app.get("/shipments/{shipment_id}")
async def get_shipment(shipment_id: int):
    try:
        sql_cursor.execute("SELECT * FROM Shipments WHERE ShipmentID = ?", (shipment_id,))
        shipment = sql_cursor.fetchone()
        if not shipment:
            raise HTTPException(status_code=404, detail="Shipment not found")

        query = """
        MATCH (s:Shipment {ID: $shipment_id})-[:HAS_ANALYTICS]->(a:Analytics)
        RETURN a.Delays as Delays, a.Efficiency as Efficiency, a.CustomerRating as CustomerRating
        """
        analytics = neo4j_graph.run(query, shipment_id=str(shipment_id)).data()

        return {
            "ShipmentID": shipment[0],
            "ShipmentName": shipment[1],
            "ParcelID": shipment[2],
            "ShipmentDate": shipment[3],
            "DeliveryDate": shipment[4],
            "Status": shipment[5],
            "CurrentLocation": shipment[6],
            "Analytics": analytics
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# API to update shipment status
@app.put("/shipments/{shipment_id}/status")
async def update_shipment_status(shipment_id: int, status: str):
    try:
        sql_cursor.execute("UPDATE Shipments SET Status = ? WHERE ShipmentID = ?", (status, shipment_id))
        sql_conn.commit()
        return {"message": "Status updated successfully"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run the FastAPI app
if __name__ == "__main__":
    import nest_asyncio
    import uvicorn

    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

INFO:     Started server process [32632]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
